Refer here for instructions to [download and run this sample locally](https://developers.arcgis.com/python/sample-notebooks/#Download-and-run-the-sample-notebooks) on your computer

# Fighting California forest fires using spatial analysis

The state of California has had a dangerous fire season in 2015 and 2016. A standard procedure while fighting these fires is identifying facilities that are at risk and evacuating them. Tasks such as this can be accomplished easily using **spatial analysis** tools available on your GIS. Spatial analysis tools allow overlaying the extent of fire and the locations of the facilities on a map and identifying the ones that fall within the fire's extent.

Thus, this sample demonstrates the application spatial analysis tools such as **buffer and overlay**.

## Data preparation
To run through this sample, we need a web feature layer with 2 layers, one representing the extent of fire and another with the locations of facilities. To accomplish this, let us publish a zipped file geodatabase (.fgdb) containing this data. To learn more about publishing, see the samples under "05 Content Publishers" section.

In [1]:
#Create a GIS connection
from arcgis import GIS

gis = GIS("https://yourorg.arcgis.com","username","password")

In [2]:
#Upload the zipped file geodatabase and add it as an item
shp_file_path = ".\data\CA_Fires_Facilities.gdb.zip"
item_properties = {'title':'California fires'}
added_item = gis.content.add(item_properties, shp_file_path)

In [3]:
#Publish as a web layer
CA_fires_item = added_item.publish()
CA_fires_item

<Item title:"California fires" type:Feature Service owner:sharing1>

## Visualize the fire data
Let us create a map widget and display the fire datasets.

In [4]:
map1 = gis.map("San Bernardino, CA", zoomlevel = 8)
map1

In [5]:
map1.add_layer(CA_fires_item)

The feature layer we just published contains two layers, the first is a point layer representing the facility locations and second is a polygon layer representing the fire boundaries. As you interact with the map widget, you can notice that a few facilities are in the vicinity of fires. The rest of the sample shows how to extract these facilities programmatically using GIS spatial analysis. 

## Create a buffer of 4 miles around fire boundaries
To minimize the risk, let us create a buffer of 4 miles around the fire boundaries and consider all facilities that fall within this are at risk. The process of buffer expands the feature's boundaries in all directions. To learn more about buffers, refer to the [tool documentation](http://desktop.arcgis.com/en/arcmap/latest/tools/analysis-toolbox/buffer.htm). To learn more about the REST API used for the tool, refer to the [API documentation](https://developers.arcgis.com/rest/analysis/api-reference/create-buffers.htm)

To perform buffers, we use the **`FeatureAnalysisTools`** class available from **`tools`** module. We access this via `featureanalysis` property of the `tools` property of your `GIS` object. 

As an input to the tool, we provide a **`FeatureLayer`** object created from the first layer of the web layer we just published. `FeatureLayer` objects can be accessed from the `layers` property of **`FeatureService`** object. A `FeatureService` object is constructed from the url of the feature service item we just published in the data preparation section. Both `FeatureLayer` and `FeatureService` can be accessed through the **`lyr`** module.

In [6]:
from arcgis.lyr import FeatureLayer, FeatureService

#Create a FeatureService object using the url property of the item
CA_fires_FS = FeatureService(CA_fires_item.url, gis)

#Query the FeatureService object's layers property to obtain both the layers
fire_boundaries_layer = CA_fires_FS.layers[1]
facility_location_layer = CA_fires_item.layers[0]

In [7]:
#Access the tools via the GIS object.
tools = gis.tools
fire_buffers = tools.featureanalysis.create_buffers(fire_boundaries_layer,
                                                    distances = [4],
                                                    units = "miles",
                                                    output_name = "California_fire_buffers")

Submitted.
Executing...


In [8]:
fire_buffers

<Item title:"California_fire_buffers" type:Feature Service owner:sharing1>

Since the `output_name` parameter was specified, the tool created a few feature service for output. We can visualize this by adding it to a new map.

In [9]:
map2 = gis.map("San Bernardino, CA", zoomlevel = 8)
map2

In [10]:
map2.add_layer(fire_buffers)

To illustrate the effect of buffers, let us add the original layers to the map. Now we can observe more facilities falling within the buffered area.

In [11]:
map2.add_layer(CA_fires_item)

## Perform overlay analysis to extract facilities that fall within the fire buffers

To programattically extract the locations that fall within the fire buffers, we use **`overlay_layers`** tool under `FeatureAnalysisTools` class just like we did for `create_buffers` tool earlier. The overlay layers tool supports a few overlay types, here we use **`Intersect`** as we need to perform a spatial intersection to identify the facilities that are located within the fire boundaries. To learn more about this operation, refer to the [documentation](https://developers.arcgis.com/rest/analysis/api-reference/overlay-layers.htm).

In [12]:
# Create a FeatureLayer object from the buffer output created in the previous step
fire_buffers_FS = FeatureService(fire_buffers.url, gis)
fire_buffers_feature_layer = fire_buffers_FS.layers[0]

#Feed the fire buffer feature layer 
overlay_result = tools.featureanalysis.overlay_layers(input_layer = fire_buffers_feature_layer,
                                                     overlay_layer = facility_location_layer,
                                                     overlay_type = "Intersect",
                                                     output_name = "At_risk_infrastructure")

Submitted.
Executing...


In [13]:
overlay_result

<Item title:"At_risk_infrastructure" type:Feature Service owner:sharing1>

Let us create another map and display only the overlay result

In [14]:
map3 = gis.map("San Bernardino, CA", zoomlevel = 8)
map3

In [15]:
map3.add_layer(overlay_result)

Let us read this result as a `FeatureLayer` and display the attribute table to get a sense of the results.

In [16]:
at_risk_facilities_FS = FeatureService(overlay_result.url, gis)
at_risk_facilities = at_risk_facilities_FS.layers[0]

at_risk_facilities.query()[0] #only the first item is displayed for brevity

{'attributes': {'acres': 10850,
  'addrln1': ' ',
  'addrln2': ' ',
  'analysisarea': 112.45956964,
  'buff_dist': 4.0,
  'cat1': 'Physical Features',
  'cat2': 'Power Plants',
  'cat3': ' ',
  'city': ' ',
  'containmen': 0.5,
  'date_updat': 1288569600000,
  'descriptio': ' ',
  'dis_status': ' ',
  'email': ' ',
  'ext_id': '1654548',
  'hours': ' ',
  'info1': ' ',
  'info2': ' ',
  'join_count': 1,
  'latitude': 34.1766729,
  'link': 'http://egis3.lacounty.gov/lms/?p=24626',
  'longitude': -117.872006,
  'name': 'Azusa Powerplant Conduit',
  'name_1': 'Fish Fire',
  'objectid': 40242905,
  'objectid_1': 1,
  'org_name': 'US Geological Survey (USGS)',
  'orig_fid': 5,
  'perimeter1': 1300,
  'perimeter_': 1466924400000,
  'phones': ' ',
  'point_x': 6600383.59242,
  'point_y': 1886699.63928,
  'post_id': 24626.0,
  'source': 'HSIP Freedom Gnis_cultural_fe',
  'state': ' ',
  'status': 'Active',
  'target_fid': 15,
  'url': ' ',
  'use_type': 'publish',
  'zip': ' '},
 'geometry': {

## Read analysis results as a pandas dataframe for analysis
The query we ran in the previous step returned a list of dictionaries representing the features. To inspect this further, let us convert it into a [pandas](http://pandas.pydata.org/) [dataframe](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [17]:
import pandas as pd

In [18]:
# Let us filter the query such that it only returns the attributes and not the geometries
query_results = at_risk_facilities.query(returnGeometry = False)

# Let us simplfy the dictionary
query_results_att = [q['attributes'] for q in query_results]

# Read it as a data frame
df1 = pd.DataFrame(query_results_att)
df1

,acres,addrln1,addrln2,analysisarea,buff_dist,cat1,cat2,cat3,city,containmen,...,point_x,point_y,post_id,source,state,status,target_fid,url,use_type,zip
0,10850,,,112.459570,4.0,Physical Features,Power Plants,,,0.50,...,6.600384e+06,1.886700e+06,24626.0,HSIP Freedom Gnis_cultural_fe,,Active,15,,publish,
1,1080,,,57.433120,4.0,Health and Mental Health,Hospitals and Medical Centers,,,0.65,...,6.369202e+06,1.880309e+06,26620.0,HSIP Freedom Gnis_structures,,Active,17,,publish,
2,8940,,,60.135538,4.0,Health and Mental Health,Hospitals and Medical Centers,,,0.80,...,6.445879e+06,1.924838e+06,27055.0,HSIP Freedom Gnis_structures,,Active,48,,publish,
3,8940,12653 Osborne St.,,60.135538,4.0,Transportation,Airports,,Pacoima,0.80,...,6.436785e+06,1.916959e+06,13559.0,Current Services Locator,CA,Active,57,,publish,91331
4,1080,5601 De Soto Ave.,,57.433120,4.0,Health and Mental Health,Hospitals and Medical Centers,911 Receiving,Woodland Hills,0.65,...,6.383286e+06,1.885131e+06,730.0,Current Services Locator,CA,Active,59,www.kaiserpermanente.org,publish,91365
5,8940,9449 San Fernando Rd.,,60.135538,4.0,Health and Mental Health,Hospitals and Medical Centers,911 Receiving,Sun Valley,0.80,...,6.441925e+06,1.910013e+06,637.0,211,CA,Active,166,www.pacificahospital.com,publish,91352
6,8940,12756 Van Nuys Blvd,,60.135538,4.0,Health and Mental Health,Hospitals and Medical Centers,,Pacoima,0.80,...,6.437589e+06,1.922409e+06,70599.0,211,CA,Active,218,www.nevhc.org/,publish,91331


From the data frame, we can see there are 7 features which fell within the fire buffers. To make this result usable, let us export a CSV with only the facility name and other critical details.

In [19]:
df1_simplified = df1[['name', 'cat1', 'post_id']]
df1_simplified

,name,cat1,post_id
0,Azusa Powerplant Conduit,Physical Features,24626.0
1,Motion Picture And Television Fund Hospital,Health and Mental Health,26620.0
2,Pacoima Memorial Hospital,Health and Mental Health,27055.0
3,Whiteman Airport,Transportation,13559.0
4,Kaiser Permanente - Woodland Hills Medical Center,Health and Mental Health,730.0
5,Pacifica Hospital Of The Valley,Health and Mental Health,637.0
6,Northeast Valley Health Corporation - Pacoima ...,Health and Mental Health,70599.0


In [20]:
# Export this to a csv file. This CSV can then be shared with fire fighters.
csv_file = r'.\data\at_risk_facilities.csv'
df1_simplified.to_csv(csv_file)

## Summary
Thus this sample demonstrated how **spatial analysis** can be easily performed using ArcGIS Python API. The first part of the sample showed how **buffers** can be used to expand the boundary of features. The second part showed how **overlay analysis** can be used to perform *point in polygon* search which allowed us to programmatically identify the facilities that fell within the risk zones. Finally the sample showed how other powerful data science libraries such as **pandas** can be used in conjunction with ArcGIS to explore the results.

Throught out the sample, the analysis tasks were performed remotely on the ArcGIS server as geoprocessing tasks instead of on the local machine which can greatly speed up the processing time for larger datasets.